In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import zipfile
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_predict, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.svm import SVC, NuSVC, LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from io import BytesIO, StringIO

In [18]:
# URL del archivo que deseas descargar
url = "https://www.kaggle.com/datasets/alphiree/cardiovascular-diseases-risk-prediction-dataset/download?datasetVersionNumber=3"

# Carpeta donde deseas guardar el archivo descomprimido
carpeta_destino = "Datasets"

# Descargar el archivo
response = requests.get(url)

# Descomprimir el archivo
zip_file = zipfile.ZipFile(BytesIO(response.content))
zip_file.extractall(carpeta_destino)


BadZipFile: File is not a zip file